# Propositional logic

In [68]:
data PropL = PVar String | PNot PropL | PropL `PAnd` PropL | PropL `POr` PropL deriving Eq

In [69]:
instance Show PropL where
    show (PVar s) = s
    show (PNot p) = "~" ++ show p
    show (p `PAnd` q) = "(" ++ show p ++ " & " ++ show q ++ ")"
    show (p `POr` q) = "(" ++ show p ++ " | " ++ show q ++ ")"

In [70]:
form1 = PVar "p1" `PAnd` PNot (PVar "p1" `POr` PVar "p3")

In [71]:
show form1

"(p1 & ~(p1 | p3))"

In [72]:
form2 = (PVar "p3" `PAnd` PVar "p3") `POr` PNot (PNot (PVar "p1") `POr` PVar "p3")

In [73]:
form2

((p3 & p3) | ~(~p1 | p3))

## Number of operators

In [74]:
opsNr :: PropL -> Int
opsNr (PVar _) = 0
opsNr (PNot p) = 1 + opsNr p
opsNr (PAnd p q) = 1 + opsNr p + opsNr q
opsNr (POr p q) = 1 + opsNr p + opsNr q

In [75]:
opsNr form1

3

```
opsNr (PVar "p1" `PAnd` PNot (PVar "p1" `POr` PVar "p3"))
= 1 + opsNr (PVar "p1) + opsNr (PNot (PVar "p1" `POr` PVar "p3))
= 1 + 0 + opsNr (PNot (PVar "p1" `POr` PVar "p3"))
= 1 + 0 + 1 + opsNr (PVar "p1" `POr` PVar "p3")
= 1 + 0 + 1 + 1 + opsNr (PVar "p1") + opsNr (PVar "p3)
= 1 + 0 + 1 + 1 + 0 + 0
= 3
```

In [76]:
depth :: PropL -> Int
depth (PVar _) = 0
depth (PNot p) = 1 + depth p
depth (PAnd p q) = 1 + max (depth p) (depth q)
depth (POr p q) = 1 + max (depth p) (depth q)

In [77]:
depth form1

3

In [78]:
depth form2

4

In [79]:
gatherNames :: PropL -> [String]                  
gatherNames (PVar s) = [s]                        
gatherNames (PNot p) = gatherNames p                
gatherNames (PAnd p q) = gatherNames p ++ gatherNames q
gatherNames (POr p q) = gatherNames p ++ gatherNames q

In [80]:
gatherNames form2

["p3","p3","p1","p3"]

In [81]:
import Data.List

propNames :: PropL -> [String]
propNames = sort . nub . gatherNames

In [82]:
propNames form2

["p1","p3"]

# Conversion to CNF

## de Morgan's (non recursive)

In [83]:
dM1 :: PropL -> PropL
dM1 (PNot (p `POr` q)) = PNot p `PAnd` PNot q
dM1 p = p

In [84]:
dM1 (PNot (PVar "p1" `POr` PVar "p2"))

(~p1 & ~p2)

In [85]:
dM1 form2

((p3 & p3) | ~(~p1 | p3))

In [86]:
dM :: PropL -> PropL
dM (PNot (p `PAnd` q)) = PNot p `POr` PNot q
dM (PNot (p `POr` q)) = PNot p `PAnd` PNot q
dM p = p

In [87]:
dM form2

((p3 & p3) | ~(~p1 | p3))

## Recursive de Morgan's

In [88]:
dMrec :: PropL -> PropL
dMrec (PNot (p `PAnd` q)) = PNot (dM p) `POr` PNot (dM q)
dMrec (PNot (p `POr` q)) = PNot (dM p) `PAnd` PNot (dM q)
dMrec (PNot p) = PNot (dM p)
dMrec (p `PAnd` q) = dM p `PAnd` dM q
dMrec (p `POr` q) = dM p `POr` dM q
dMrec (PVar p) = PVar p

In [90]:
form2

((p3 & p3) | ~(~p1 | p3))

In [91]:
dMrec form2

((p3 & p3) | (~~p1 & ~p3))

## Recursive DNE

In [45]:
dne :: PropL -> PropL
dne (PNot (PNot p)) = dne p
dne (PNot p) = PNot (dne p)
dne (p `PAnd` q) = dne p `PAnd` dne q 
dne (p `POr` q) = dne p `POr` dne q 
dne (PVar p) = PVar p

In [46]:
toCNF :: PropL -> PropL
toCNF = dne . dMrec

In [47]:
toCNF form2

((p3 & p3) | (p1 & ~p3))

## Distributive Law

In [48]:
distLaw :: PropL -> PropL 
distLaw ((p `PAnd` q) `POr` (r `PAnd` s)) = (distLaw p `POr` distLaw r) `PAnd` (distLaw p `POr` distLaw s) `PAnd` (distLaw q `POr` distLaw r) `PAnd` (distLaw q `POr` distLaw s) -- double distributivity
distLaw (p `POr` (q `PAnd` r)) = (distLaw p `POr` distLaw q) `PAnd` (distLaw p `POr` r) --left dist
distLaw ((q `PAnd` r) `POr` p) = (distLaw q `POr` distLaw p) `PAnd` (distLaw r `POr` distLaw p) --right dist
distLaw (PNot p) = PNot (distLaw p)
distLaw (p `PAnd` q) = distLaw p `PAnd` distLaw q
distLaw (p `POr` q) = distLaw p `POr` distLaw q
distLaw (PVar p) = PVar p

In [49]:
distLaw form2

((p3 | ~(~p1 | p3)) & (p3 | ~(~p1 | p3)))

In [54]:
toCNF = distLaw . dne . dM

In [55]:
toCNF form2

((p3 | ~(~p1 | p3)) & (p3 | ~(~p1 | p3)))

## Computing truth tables

In [94]:
type VarAssignment = [(String,Bool)]
type TruthTable = [(VarAssignment,Bool)]

In [ ]:
toTruthTable :: PropL -> TruthTable
toTruthTable = undefined

assign1 = [("p",True),("q",False)]

      p & q
p q |
1 1 |  1
1 0 |  0
0 1 |  0
0 0 |  0